Import necessary Python packages.

In [108]:
import pandas as pd
import numpy as np

Read data .csv file and store in a pandas DataFrame.

In [109]:
data = pd.read_csv('../data/project-filtered-data.csv')

Inspect the attributes in the data.

In [110]:
print(f'data columns: {data.columns.to_list()}')

data columns: ['case_month', 'res_state', 'state_fips_code', 'res_county', 'county_fips_code', 'age_group', 'sex', 'race', 'ethnicity', 'case_positive_specimen_interval', 'case_onset_interval', 'process', 'exposure_yn', 'current_status', 'symptom_status', 'hosp_yn', 'icu_yn', 'death_yn', 'underlying_conditions_yn']


Only include the following attributes in the data and drop all others:
  - case_month
  - res_state
  - state_fips_code
  - age_group
  - sex
  - race
  - ethnicity
  - current_status
  - symptom_status
  - hosp_yn
  - icu_yn
  - death_yn
  - underlying_conditions_yn

In [111]:
data = data[['case_month', 'res_state', 'state_fips_code', 'age_group',\
    'sex', 'race', 'ethnicity', 'current_status', 'symptom_status', 'hosp_yn',\
    'icu_yn', 'death_yn', 'underlying_conditions_yn']]
print(f'data columns: {data.columns.to_list()}')

data columns: ['case_month', 'res_state', 'state_fips_code', 'age_group', 'sex', 'race', 'ethnicity', 'current_status', 'symptom_status', 'hosp_yn', 'icu_yn', 'death_yn', 'underlying_conditions_yn']


Check for na/nan values.

In [112]:
pd.DataFrame.from_dict(data={
    'any_na' : data.isna().any(),
    'num_na' : data.isna().sum(),
    'percent_na' : (data.isna().sum() / data.shape[0]) * 100
})

,any_na,num_na,percent_na
case_month,False,0,0.000000
res_state,False,0,0.000000
state_fips_code,False,0,0.000000
age_group,False,0,0.000000
sex,False,0,0.000000
race,True,39143,8.090957
ethnicity,True,43783,9.050057
current_status,False,0,0.000000
symptom_status,False,0,0.000000
hosp_yn,False,0,0.000000


Deep clone data into variable data2 and drop na/nan values from data2.

In [113]:
data2 = data.copy(deep=True)
data2.dropna(inplace=True)

print(f'data number of observations: {data.shape[0]}')
print(f'data2 number of observations: {data2.shape[0]}')
print(f'difference between data and data2 number of observations: {data.shape[0] - data2.shape[0]}')

data number of observations: 483787
data2 number of observations: 440004
difference between data and data2 number of observations: 43783


Now, check data types.

In [114]:
data_types_df = pd.DataFrame().from_dict(data={
    'data_type' : data2.dtypes
})
data_types_df

,data_type
case_month,object
res_state,object
state_fips_code,int64
age_group,object
sex,object
race,object
ethnicity,object
current_status,object
symptom_status,object
hosp_yn,object


For every attribute of type 'object' in data2, make sure there are no 'Missing' or 'nul' values.

In [115]:
object_types_df = data_types_df[data_types_df['data_type'] == 'object']
object_types_df

,data_type
case_month,object
res_state,object
age_group,object
sex,object
race,object
ethnicity,object
current_status,object
symptom_status,object
hosp_yn,object
icu_yn,object


In [116]:
missing_nul_cols = []
for col in object_types_df.index:
    if 'Missing' in data[col].unique().tolist() or 'nul' in data[col].unique().tolist():
        missing_nul_cols.append(col)

print(f'columns with missing or nul values: {missing_nul_cols}')

columns with missing or nul values: ['race', 'ethnicity', 'symptom_status', 'icu_yn']


Erase all 'Missing' or 'nul' records from the data.

In [117]:
for col in missing_nul_cols:
    data2 = data2[(data2[col] != 'Missing') & (data2[col] != 'nul')]

print(f'data number of observations: {data.shape[0]}')
print(f'data2 number of observations: {data2.shape[0]}')
print(f'difference between data and data2 number of observations: {data.shape[0] - data2.shape[0]}')

data number of observations: 483787
data2 number of observations: 428867
difference between data and data2 number of observations: 54920


Make sure there are no duplicates.

In [118]:
data2[data2.duplicated()].count()

case_month                  405399
res_state                   405399
state_fips_code             405399
age_group                   405399
sex                         405399
race                        405399
ethnicity                   405399
current_status              405399
symptom_status              405399
hosp_yn                     405399
icu_yn                      405399
death_yn                    405399
underlying_conditions_yn    405399
dtype: int64

There is an overwhelming portion of the data set that has been flagged as being duplicated. Such a common occurrence of *truly* duplicate data is highly unlikely. However, we don't have any means of verifying which are actually duplicates and which are just patients with very similar backgrounds, based on the available attributes, as there are no unique IDs or identifying keys in the data to distinguish between different COVID-19 cases.

**Therefore, we will assume that the original CDC data set does *not* contain *any* duplicate records, i.e., counting a single COVID-19 case more than one time**.

Assign data2 to data variable.

In [119]:
data = data2
print(f'data number of observations: {data.shape[0]}')

data number of observations: 428867


Save the data as a csv file.

In [120]:
data.to_csv(r'../data/trimmed-data-with-unknowns.csv', index=False)

Now, we should create an alternate data set that does **not** include 'Unknown' values.

First, let's deep clone data to data2.

In [121]:
data2 = data.copy(deep=True)

Next, let's get the data types for data2 and create a DataFrame that only includes 'object' type data.

In [122]:
data_types_df = pd.DataFrame().from_dict(data={
    'data_type' : data2.dtypes
})

In [123]:
object_types_df = data_types_df[data_types_df['data_type'] == 'object']
object_types_df

,data_type
case_month,object
res_state,object
age_group,object
sex,object
race,object
ethnicity,object
current_status,object
symptom_status,object
hosp_yn,object
icu_yn,object


Now, let's determine which attributes in data2 contain Unknown values.

In [124]:
unknown_cols = []
for col in object_types_df.index:
    if 'Unknown' in data2[col].unique().tolist():
        unknown_cols.append(col)

print(f'columns with unknown values: {unknown_cols}')

columns with unknown values: ['race', 'ethnicity', 'symptom_status']


At this point, we should drop all the records with Unknown values from data2.

In [125]:
for col in unknown_cols:
    data2 = data2[data2[col] != 'Unknown']

print(f'data number of observations: {data.shape[0]}')
print(f'data2 number of observations: {data2.shape[0]}')
print(f'difference between data and data2 number of observations: {data.shape[0] - data2.shape[0]}')

data number of observations: 428867
data2 number of observations: 396603
difference between data and data2 number of observations: 32264


Lastly, we should export data2 as a csv file.

In [126]:
data2.to_csv(r'../data/trimmed-data-without-unknowns.csv', index=False)